In [ ]:
pip install tabulate

In [1]:
import torch
import os
from transformers import AutoTokenizer
from llmcompressor.transformers import SparseAutoModelForCausalLM
import matplotlib.pyplot as plt
import numpy as np

model_stub = "neuralmagic/Meta-Llama-3.1-8B-FP8"
tokenizer = AutoTokenizer.from_pretrained(model_stub)

# Load the model
model = SparseAutoModelForCausalLM.from_pretrained(model_stub, torch_dtype=torch.float16)

weights = model.state_dict()

# # Extract weights for attention and MLP layers
# selfattn_weights = {}
attention_weights = {}
mlp_weights = {}

/Users/roshniramesh/fp8llm/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.53it/s]
Decompressing model: 224it [00:20, 10.79it/s]

2024-09-24T16:03:24.635667-0400 | infer_recipe_from_model_path | INFO - Found recipe in the model_path: /Users/roshniramesh/.cache/huggingface/hub/models--neuralmagic--Meta-Llama-3.1-8B-FP8/snapshots/91994a9a9b33939a9cc0a15b8f2a1f5aafb68aef/recipe.yaml
2024-09-24T16:03:24.636568-0400 | _check_create_state | INFO - State created for compression lifecycle
2024-09-24T16:03:24.640208-0400 | create_instance | INFO - Loading recipe from file /Users/roshniramesh/.cache/huggingface/hub/models--neuralmagic--Meta-Llama-3.1-8B-FP8/snapshots/91994a9a9b33939a9cc0a15b8f2a1f5aafb68aef/recipe.yaml


2024-09-24T16:03:24.695501-0400 | _check_compile_recipe | INFO - Recipe compiled and 1 modifiers created


manager stage: Model structure initialized


2024-09-24T16:03:24.698864-0400 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 1 modifiers
2024-09-24T16:03:24.699185-0400 | initialize_recipe | INFO - Applied an unstaged recipe to the model


In [2]:
temp = {}
for name, param in weights.items():
    if any(layer in name for layer in ['.0', '15', '31']) and (
        name.endswith('self_attn.q_proj.weight') or 
        name.endswith('self_attn.k_proj.weight') or 
        name.endswith('self_attn.v_proj.weight') or 
        name.endswith('self_attn.o_proj.weight')
    ):
        # temp[name] = param.to(dtype=torch.float8_e4m3fn)
        temp[name] = param
        print(temp[name].dtype)

torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn


In [ ]:
print(temp)

In [ ]:
print(temp.keys())

In [ ]:
zerolayer = {name: weight for name, weight in temp.items() if any(x in name for x in ["0"])}
print(zerolayer.keys())

In [ ]:
fiflayer = {name: weight for name, weight in temp.items() if any(x in name for x in ["15"])}
print(fiflayer.keys())

In [ ]:
thirlayer = {name: weight for name, weight in temp.items() if any(x in name for x in ["31"])}
print(thirlayer.keys())

In [ ]:
colors = ['r', 'g', 'b', 'c']
plt.figure(figsize=(5, 5))

num_bins = 10
bar_width = 0.2  

x_positions = np.arange(num_bins)

for i, (layer_name, weight) in enumerate(thirlayer.items()):
    if weight.dtype == torch.float8_e4m3fn:
        weight = weight.to(torch.float32)
    weight_np = weight.detach().cpu().numpy()

    hist, bin_edges = np.histogram(weight_np.flatten(), bins=num_bins)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    plt.bar(x_positions + i * bar_width, hist, width=bar_width,
            alpha=0.5, label=layer_name, color=colors[i % len(colors)])

tick_positions = x_positions + bar_width * (len(thirlayer) - 1) / 2

plt.xticks(tick_positions, 
           [f'{bin_edges[j]:.2f} - {bin_edges[j+1]:.2f}' for j in range(num_bins)], 
           rotation=45, ha='right')

plt.title('Weight Distributions of Layer 31')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
colors = ['r', 'g', 'b', 'c']
plt.figure(figsize=(5, 5))

num_bins = 10
bar_width = 0.2  

x_positions = np.arange(num_bins)

for i, (layer_name, weight) in enumerate(fiflayer.items()):
    if weight.dtype == torch.float8_e4m3fn:
        weight = weight.to(torch.float32)
    weight_np = weight.detach().cpu().numpy()

    hist, bin_edges = np.histogram(weight_np.flatten(), bins=num_bins)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    plt.bar(x_positions + i * bar_width, hist, width=bar_width,
            alpha=0.5, label=layer_name, color=colors[i % len(colors)])

tick_positions = x_positions + bar_width * (len(fiflayer) - 1) / 2

plt.xticks(tick_positions, 
           [f'{bin_edges[j]:.2f} - {bin_edges[j+1]:.2f}' for j in range(num_bins)], 
           rotation=45, ha='right')

# Customize the plot
plt.title('Weight Distributions of Layer 15')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
colors = ['r', 'g', 'b', 'c']
plt.figure(figsize=(5, 5))

num_bins = 10
bar_width = 0.2 

x_positions = np.arange(num_bins)

for i, (layer_name, weight) in enumerate(zerolayer.items()):
    if weight.dtype == torch.float8_e4m3fn:
        weight = weight.to(torch.float32)
    weight_np = weight.detach().cpu().numpy()

    hist, bin_edges = np.histogram(weight_np.flatten(), bins=num_bins)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    # plt.xlim([bin_edges[0], bin_edges[-1]])
    plt.bar(x_positions + i * bar_width, hist, width=bar_width,
            alpha=0.5, label=layer_name, color=colors[i % len(colors)])

tick_positions = x_positions + bar_width * (len(zerolayer) - 1) / 2

plt.xticks(tick_positions, 
           [f'{bin_edges[j]:.2f} - {bin_edges[j+1]:.2f}' for j in range(num_bins)], 
           rotation=45, ha='right')

# Customize the plot
plt.title('Weight Distributions of Layer 0')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
print(temp['model.layers.0.self_attn.q_proj.weight'])

In [ ]:
print(temp['model.layers.0.self_attn.q_proj.weight'].shape)
print(temp['model.layers.0.self_attn.q_proj.weight'].size)



In [ ]:

# def extract_exp_mantissa(fp8_tensor):
#     # Ensure the tensor is of float8_e4m3fn type
#     if fp8_tensor.dtype != torch.float8_e4m3fn:
#         raise ValueError("Input tensor must be of type torch.float8_e4m3fn")

#     int_repr = fp8_tensor.view(torch.int8)

#     # Masks for extracting mantissa and exponent
#     mantissa_mask = 0b00000111  # 3 bits mask for mantissa
#     exp_mask = 0b00001111       # 4 bits mask for exponent

#     # Initialize lists to store extracted mantissas and exponents
#     # mantissas = []
#     # exponents = []
#     max_mantissa = float('-inf')
#     corresponding_exponent = None

#     # Iterate through each value in the tensor
#     for value in int_repr.flatten():
#         val = value.item()
#         print(val)

#         # Extract mantissa and exponent
#         mantissa = val & mantissa_mask
#         exp = (val >> 3) & exp_mask
#         # mantissas.append(mantissa)
#         # exponents.append(exp)

#         if mantissa > max_mantissa:
#             max_mantissa = mantissa
#             corresponding_exponent = exp
        
#         # mantissas.append(mantissa)
#         # exponents.append(exp)

#         # Print the details (optional)
#         binary_str = format(val & 0b11111111, '08b')  # Mask to 8 bits
#         # return exp, mantissa
#         print(f"Value: {val} | Binary: {binary_str} | Mantissa: {mantissa} | Exp: {exp}")

#     # maxMan= max(mantissas)
#     # maxExp = exponents[mantissas.index(maxMan)]
#     # print(mantissas)
#     # print(exponents)
            
#     return max_mantissa, corresponding_exponent
#     # return torch.tensor(mantissas), torch.tensor(exponents)

# # Example usage
# # fp8_tensor = torch.tensor(temp['model.layers.0.self_attn.q_proj.weight'], dtype=torch.float8_e4m3fn)  # Sample FP8 tensor
# # mantissas, exponents = extract_exp_mantissa(fp8_tensor)
# # print(f"Mantissas: {mantissas}, Exponents: {exponents}")
# test = torch.tensor([-0.108, 1.44, 0, 0.40625], dtype=torch.float8_e4m3fn)
# mantissa, expo = extract_exp_mantissa(test)
# print(mantissa)
# print(expo)



In [35]:
def extract_exp_mantissa(fp8_tensor):
    # Ensure the tensor is of float8_e4m3fn type
    if fp8_tensor.dtype != torch.float8_e4m3fn:
        raise ValueError("Input tensor must be of type torch.float8_e4m3fn")

    int_repr = fp8_tensor.view(torch.int8)
    int_repr = torch.flatten(int_repr, start_dim=0)


    # Masks for extracting mantissa and exponent
    mantissa_mask = 0b00000111  # 3 bits mask for mantissa
    exp_mask = 0b00001111       # 4 bits mask for exponent

    # Extract mantissa and exponent using bitwise operations
    mantissas = int_repr & mantissa_mask
    exponents = (int_repr >> 3) & exp_mask

    # Find the maximum mantissa and the corresponding exponent
    # torch.flatten(fp8_tensor)
    # int_repr = torch.flatten(int_repr, start_dim=0)
    # mantissas = torch.flatten(mantissas, start_dim=0)
    # exponents = torch.flatten(exponents, start_dim=0)
    
    # print(f"int_rep: {int_repr.shape}\tmantissas: {mantissas.shape}\texponents:{exponents.shape}")
    print(mantissas)
    print(exponents)
    max_mantissa, max_idx = torch.max(mantissas, dim=0)  # Use dim=0 for 1D result across columns
    # max_mantissa = max_mantissa.max(dim=0)
    corresponding_exponent = exponents[max_idx]


    return max_mantissa, corresponding_exponent  

# test = torch.tensor([-0.108, 1.44, 0, 0.40625], dtype=torch.float8_e4m3fn)
# mantissa, expo = extract_exp_mantissa(test)
# print(mantissa)
# print(expo)



In [ ]:
weights = temp['model.layers.0.self_attn.q_proj.weight']
weights = weights[0:16,0:16]
print(weights)

In [36]:
def max_convolution(weights, window_size=(16, 16)):
    w_height, w_width = weights.shape
    k_height, k_width = window_size

    # Initialize an output tensor to store the max mantissas
    output_shape = (w_height - k_height + 1, w_width - k_width + 1)
    max_mantissas_output = torch.zeros(output_shape, dtype=torch.int8)
    cor_exp_output = torch.zeros(output_shape, dtype=torch.int8)  # If you plan to use it later

    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            # Define the current window
            current_window = weights[i:i + k_height, j:j + k_width]
            
            # Extract mantissas from the current window
            max_mantissa, exp = extract_exp_mantissa(current_window)
            print(max_mantissa)
            # print(max_mantissa)
            # print(exp)
            # max_val, max_ind = max_mantissa.max(dim=0)  # Max along the correct dimension
            
            # Store the max mantissa in the output tensor
            max_mantissas_output[i, j] = max_mantissa.item()

            cor_exp_output[i, j] = exp.item()  # Get the corresponding exponent
            # x = x+1

    return max_mantissas_output, cor_exp_output

# Example usage
weights = temp['model.layers.0.self_attn.q_proj.weight']
# weights = weights[0:16, 0:16]

window_size = (16, 16)  # Define your window size
mantissas,exps = max_convolution(weights, window_size)

# Save results
mantissas_flat = mantissas.numpy().flatten()
exps_flat = exps.numpy().flatten()

with open('result.txt', 'w') as f:
    for value, exp in zip(mantissas_flat, exps_flat):  # Use zip to iterate over both arrays
        f.write(f"{exp} {value}\n")  # Write each value on a new line

tensor([3, 0, 3, 1, 2, 6, 0, 1, 5, 6, 6, 0, 2, 1, 5, 3, 7, 1, 3, 5, 4, 2, 0, 1,
        4, 1, 2, 1, 1, 4, 4, 0, 1, 2, 1, 2, 1, 0, 0, 1, 0, 3, 1, 0, 2, 0, 4, 1,
        1, 0, 1, 4, 7, 3, 0, 0, 1, 1, 3, 0, 1, 0, 1, 1, 6, 3, 2, 5, 1, 6, 0, 1,
        2, 6, 0, 1, 3, 0, 6, 1, 1, 4, 4, 6, 0, 7, 0, 1, 4, 2, 1, 1, 3, 4, 4, 1,
        3, 3, 5, 3, 3, 0, 0, 0, 4, 0, 0, 2, 1, 3, 6, 0, 0, 2, 6, 3, 0, 4, 0, 1,
        6, 7, 1, 0, 0, 4, 5, 0, 0, 0, 0, 4, 6, 4, 0, 1, 4, 3, 1, 1, 2, 5, 1, 2,
        3, 0, 6, 5, 1, 7, 0, 0, 6, 3, 1, 1, 1, 5, 7, 2, 3, 1, 0, 5, 1, 4, 0, 1,
        0, 3, 0, 1, 0, 3, 0, 1, 1, 4, 1, 0, 4, 3, 0, 1, 3, 1, 0, 0, 2, 0, 2, 1,
        2, 4, 0, 2, 0, 1, 0, 2, 2, 7, 5, 1, 2, 2, 3, 1, 2, 0, 2, 3, 0, 0, 0, 2,
        1, 6, 1, 2, 3, 5, 3, 2, 1, 4, 1, 1, 6, 6, 0, 0, 1, 5, 0, 1, 1, 3, 1, 0,
        0, 3, 4, 1, 1, 2, 0, 2, 3, 3, 1, 2, 3, 3, 1, 2], dtype=torch.int8)
tensor([0, 2, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
        1, 0, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 0

In [ ]:
import torch
from tabulate import tabulate
 
f32_type = torch.float32
bf16_type = torch.bfloat16
e4m3_type = torch.float8_e4m3fn
e5m2_type = torch.float8_e5m2

# collect finfo for each type
table = []
for dtype in [f32_type, bf16_type, e4m3_type, e5m2_type]:
    numbits = 32 if dtype == f32_type else 16 if dtype == bf16_type else 8
    info = torch.finfo(dtype)
    table.append([info.dtype, numbits, info.max, 
                  info.min, info.smallest_normal, info.eps])

headers = ['data type', 'bits', 'max', 'min', 'smallest normal', 'eps']
print(tabulate(table, headers=headers))
 
'''
Output:

data type      bits          max           min  smallest normal          eps
-------------  ----  -----------  ------------  ---------------  -----------
float32          32  3.40282e+38  -3.40282e+38      1.17549e-38  1.19209e-07
bfloat16         16  3.38953e+38  -3.38953e+38      1.17549e-38    0.0078125
float8_e4m3fn     8          448          -448         0.015625        0.125
float8_e5m2       8        57344        -57344      6.10352e-05         0.25
'''